# Examining Behavior of SPX Options on Expiration Day

### 1) Data Cleaning and Preparation
- Consolidate intraday minutely data from daily SPX yahoo pulls
- Consolidate VIX intraday prices from daily VIX options files
- Consolidate 0 DTE expiration options from daily SPX options pulls
- Retrieve live minutely data from Alphavantage

### 2) Initial Check of SPX Spot Behavior Intraday
- Test for random walk
    - Check 1 min, 5 min, 10 min, 15 min, 20 min, 30 min intervals on:
        - Entire day
        - Segmented times in the day, e.g., hourly behavior, half-hour behavior, etc.
    - Perform DF test
    - Calculate probability of price movements using volatility windows and simulating price behavior

### 3) Relationship of VIX Intraday to 0 DTE Option IV
- Check if VIX and IV use calendar or trading days to annualize
    - **IV on the options chain is in calendar years with 24 hour days -> minutes_remaining/(60 x 24 x 365.25)**
- Determine Function of Skew using 3rd and 4th moments
- Plot IV surface against strikes and time of day
- Check behavior of IV movement given VIX term structure is in Contango or Backwardation

### 4) Expiration Day Option Pricing
- Implied Binomial Pricing model using probability inferred from option prices
- Monte Carlo using implied binomial probabilities then backing out he IV from simulated option prices
- Use Black Scholes but IV based on VIX intraday or HV from intraday SPX
- **Track the IV live on the option chain in IB and use BS to calculate the delta to confirm validity**


### 5) 1 DTE Overnight Options Short Vol
- Test buckets of VIX brackets and put spreads from 1 hour, 2 hour, or 3 hours before market close and premium the next day
- Test overnight volatility vs intraday volatility of SPX during high IV and low IV
- Test intraday VIX correlation with SPX and trends
- Use ARIMA for trend significance throughout the day
- Use ADF for mean reversion significance throughout the day
- Infer (r - q) from ES and SPX spot prices and see whether they conform to options prices of SPX and ES

##### Importing Necessary Modules

In [33]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pandas.stats.moments as st
import statsmodels.tsa.stattools as ts
from pandas_datareader.data import Options
pd.options.display.float_format = '{:,.4f}'.format
import statsmodels.api as sm

import numpy as np
import datetime as dt
import time
import matplotlib.pyplot as plt

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline

from scipy.stats import norm as norm

import os
os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\RetiredModules')
from helpers import maturities

os.chdir('D:\Options Data\IB Intraday')

from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
from statsmodels.graphics.tsaplots import plot_acf


##### 1) Data Preparation

In [7]:
# Retrieving Data for VIX Term Structure
v1 = pd.read_csv('https://www.quandl.com/api/v3/datasets/CHRIS/CBOE_VX1.csv?api_key=dzmzEExntfap7SNx5p6t', index_col = 0)[['Open','Close']].sort_index()
v2 = pd.read_csv('https://www.quandl.com/api/v3/datasets/CHRIS/CBOE_VX2.csv?api_key=dzmzEExntfap7SNx5p6t', index_col = 0)[['Open','Close']].sort_index()
vix = pd.read_csv('http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv', skiprows=[0], index_col = 0)
vix.index = pd.to_datetime(vix.index)
vix_term = vix.join(v1.join(v2, lsuffix = '_F1', rsuffix = '_F2').dropna()).dropna()
vix_term['Front_Weights'] = np.nan
vix_term['Back_Weights'] = np.nan

for index, row in vix_term.iterrows():
    weights = maturities(index.date())
    front_weight = weights[0]
    back_weight = weights[1]
    vix_term.loc[index, 'Front_Weights'] = front_weight
    vix_term.loc[index, 'Back_Weights'] = back_weight
    
vix_term = vix_term[list(filter(lambda x: 'Open' in x or 'Weight' in x, vix_term.columns.tolist()))]
vix_term['Contango_Ratio'] = vix_term.Front_Weights*(vix_term['VIX Open']/vix_term.Open_F1) + vix_term.Back_Weights*(vix_term.Open_F1/vix_term.Open_F2)

spx_intraday = pd.read_csv('index_intraday_1year.csv', index_col = 0)
spx_intraday.index = pd.to_datetime(spx_intraday.index)

##### 2) Initial Check of SPX Spot Behavior Intraday
- Test for random walk
    - Check 1 min, 5 min intervals on:
        - Entire day
        - Segmented times in the day, e.g., hourly behavior, half-hour behavior, etc.
    - Perform DF test
    - Calculate probability of price movements using volatility windows and simulating price behavior

**ADF test for random walk** <br>
- The test statistic is a negative number and thus in order to be significant beyond the critical values, the number must be more negative than these values, i.e. less than the critical values.
- The ADF null hypothesis is that the process is a random walk and thus non mean-reverting (i.e., a high p-value indicates that we fail to reject the null that it is a random walk)
- The output of the Augmented Dickey-Fuller test over a period is as follows:
    - The first value is the calculated test-statistic
    - The second value is the p-value.
    - The fourth is the number of data points in the sample. 
    - The fifth value, the dictionary, contains the critical values of the test-statistic at the 1, 5 and 10 percent values respectively.

**Hurst Exponent Test for Stationarity** <br>
A time series can then be characterised in the following manner:
- H < 0.5 - The time series is mean reverting
- H = 0.5 - The time series is a Geometric Brownian Motion
- H > 0.5 - The time series is trending

**Autocorrelation is highest most often during the times of 9:30 AM to 10:00 AM and 1:00 PM to 2:00 PM** <br>
**Non-mean reverting series occurs most often at 9:30 AM to 10:00 AM and from 1:00 PM to 3:00 PM**

In [160]:
def get_block_stats(curr_block, block_label):

    curr_spx_trend = curr_block.close_spx - curr_block.reset_index(drop = True).loc[0,'close_spx']
    curr_vix_trend = curr_block.close_vix - curr_block.reset_index(drop = True).loc[0,'close_vix']
    spx_X = curr_spx_trend.index ## X usually means our input variables (or independent variables)
    spx_Y = curr_spx_trend ## Y usually means our output/dependent variable

    vix_X = curr_vix_trend.index ## X usually means our input variables (or independent variables)
    vix_Y = curr_vix_trend ## Y usually means our output/dependent variable

    # # Note the difference in argument order
    spx_ols = sm.OLS(spx_Y, spx_X).fit().summary()
    spx_ols_r2 = float(pd.read_html(spx_ols.tables[0].as_html())[0].loc[1,3])
    spx_ols_coeff = float(pd.read_html(spx_ols.tables[1].as_html())[0].loc[1,1])

    vix_ols = sm.OLS(vix_Y, vix_X).fit().summary()
    vix_ols_r2 = float(pd.read_html(vix_ols.tables[0].as_html())[0].loc[1,3])
    vix_ols_coeff = float(pd.read_html(vix_ols.tables[1].as_html())[0].loc[1,1])

    spx_bar_std = np.std(curr_block.close_spx/curr_block.open_spx)
    vix_bar_std = np.std(curr_block.close_vix/curr_block.open_vix)

    spx_block_std = np.std((curr_block.close_spx.shift(1)/curr_block.close_spx).dropna())
    vix_block_std = np.std((curr_block.close_vix.shift(1)/curr_block.close_vix).dropna())

    spx_adf_sig = ts.adfuller(np.log(curr_block.close_spx))[1]
    vix_adf_sig = ts.adfuller(np.log(curr_block.close_vix))[1]

    spx_block_return = curr_block.loc[len(curr_block) - 1, 'close_spx']/curr_block.loc[0, 'close_spx'] - 1
    vix_block_return = curr_block.loc[len(curr_block) - 1, 'close_vix']/curr_block.loc[0, 'close_vix'] - 1
    
    spx_vix_corr = np.corrcoef(curr_block.close_spx,curr_block.close_vix)[0,1]
    
    block_stats = pd.DataFrame({'Block_Label': block_label,
                                'SPX_OLS_Coeff': spx_ols_coeff, 
                                'SPX_OLS_R2': spx_ols_r2,
                                'SPX_bar_std': spx_bar_std,
                                'SPX_block_std': spx_block_std,
                                'SPX_adf_sig': spx_adf_sig,
                                'SPX_block_return': spx_block_return,
                                'VIX_OLS_Coeff': vix_ols_coeff, 
                                'VIX_OLS_R2': vix_ols_r2,
                                'VIX_bar_std': vix_bar_std,
                                'VIX_block_std': vix_block_std,
                                'VIX_adf_sig': vix_adf_sig,
                                'VIX_block_return': vix_block_return,
                                'SPX_VIX_corr': spx_vix_corr,
                                'VIX_open': curr_block.loc[0,'close_vix']}, index = [0])
    return block_stats

def day_block_stats(curr_data):
    block_label = 1
    day_timeblock_stats = []
    for time_block in range(0,391,30):
        if time_block != 390:
            curr_block = curr_data.loc[time_block:time_block + 30, :].reset_index(drop = True)
            day_timeblock_stats.append(get_block_stats(curr_block, "Time{}".format(block_label)))
        block_label += 1

    day_timeblock_stats = pd.concat(day_timeblock_stats, axis = 0).reset_index(drop = True)


    curr_vix_term = vix_term[vix_term.index.date == curr_block.loc[0,'date'].date()].reset_index()

    day_timeblock_stats['VIX_level'] = curr_vix_term.loc[0,'VIX Open']
    day_timeblock_stats['Contango_Ratio'] = curr_vix_term.loc[0,'Contango_Ratio']
    day_timeblock_stats['SPX_next_block_return'] = day_timeblock_stats.SPX_block_return.shift(-1)
    day_timeblock_stats = day_timeblock_stats.dropna()
    return day_timeblock_stats

In [161]:
timeblocks_df = []

for day in spx_intraday.Date.drop_duplicates().tolist():
    curr_data = spx_intraday[spx_intraday.Date == day].reset_index()
#     curr_data[['close_spx','close_vix']].plot(figsize = (20,10), secondary_y = 'close_vix',
#                                           title = day)
#     plt.show()
    try:
        timeblocks_df.append(day_block_stats(curr_data))
    except:
        continue



In [333]:
#timeblocks_df = pd.concat(timeblocks_df,axis = 0).reset_index(drop = True)#.to_csv('spx_intraday_testing.csv')
timeblocks_df['SPX_trend'] = timeblocks_df['SPX_OLS_Coeff']*timeblocks_df['SPX_OLS_R2']
timeblocks_df['VIX_trend'] = timeblocks_df['VIX_OLS_Coeff']*timeblocks_df['VIX_OLS_R2']

In [364]:
exclude_x = ['Block_Label', 'SPX_OLS_Coeff', 'SPX_OLS_R2', 'VIX_OLS_Coeff', 'VIX_OLS_R2', 'SPX_next_block_return']

reg_estimates = []
for i in range(1,13):
    time_block_string = 'Time' + str(i)
    timeblocks_ml = timeblocks_df[timeblocks_df.Block_Label == time_block_string].iloc[:,1:]#.join(pd.get_dummies(timeblocks_df['Block_Label'])).dropna()
    timeblocks_ml_X = timeblocks_ml[list(filter(lambda x: x not in exclude_x, timeblocks_df.columns.tolist()))]
    timeblocks_ml_Y = timeblocks_ml['SPX_next_block_return']

    keep_columns = []
    p_thresh = 0.05
    for j in range(len(timeblocks_ml_X.columns)):
        timeblocks_ml_X.iloc[:,j]
        curr_reg = sm.OLS(timeblocks_ml_Y, timeblocks_ml_X.iloc[:,j]).fit().summary()
        curr_reg = pd.read_html(curr_reg.tables[1].as_html())[0]
        curr_coeff = float(curr_reg.iloc[1,1])
        curr_p = float(curr_reg.iloc[1,4])
        if curr_p < p_thresh:
            keep_columns.append(timeblocks_ml_X.iloc[:,j].name)

    if len(keep_columns) > 0:
        curr_time_block = dt.datetime.today().replace(hour = 9, minute = 30, second = 0, microsecond = 0) + dt.timedelta(minutes = 30*(i-1))
        
        #print(time_block_string)
        new_reg = sm.OLS(timeblocks_ml_Y, timeblocks_ml_X[keep_columns]).fit().summary()
        new_reg = pd.read_html(new_reg.tables[1].as_html())[0][[0,4]].dropna().set_index(0).apply(pd.to_numeric)
        new_reg = new_reg.sort_values(4)
        while new_reg.reset_index().iloc[-1,1] > 0.05:
            keep_columns = new_reg.head(len(new_reg) - 1).index.tolist()
            new_reg = sm.OLS(timeblocks_ml_Y, timeblocks_ml_X[keep_columns]).fit().summary()
            new_reg = pd.read_html(new_reg.tables[1].as_html())[0][[0,4]].dropna().set_index(0).apply(pd.to_numeric)
            new_reg = new_reg.sort_values(4)
        
        if len(keep_columns) > 0:
            new_reg = sm.OLS(timeblocks_ml_Y, timeblocks_ml_X[keep_columns]).fit().summary()
            new_reg = pd.read_html(new_reg.tables[1].as_html())[0].iloc[:,[0,1,4]].dropna().set_index(0)
            new_reg.index.name = curr_time_block.strftime("%H:%M")
            new_reg.columns = ['Coeff','P-Val']
            new_reg['Block_Label'] = curr_time_block.strftime("%H:%M")#time_block_string
            reg_estimates.append(new_reg)
reg_estimates = pd.concat(reg_estimates, axis = 0)
reg_estimates

,Coeff,P-Val,Block_Label
VIX_trend,0.0262,0.000,09:30
VIX_block_return,-0.0214,0.002,09:30
VIX_block_std,0.1781,0.049,09:30
VIX_bar_std,-0.2913,0.026,09:30
SPX_trend,0.0019,0.004,11:00
VIX_bar_std,0.1192,0.001,11:00
SPX_block_return,0.1374,0.012,11:30
VIX_trend,0.0112,0.039,12:00
SPX_block_return,-0.2031,0.001,12:30
SPX_bar_std,-2.8143,0.001,12:30
